In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np

from geopy.geocoders import Nominatim
from shapely.geometry import Point

from geopy.extra.rate_limiter import RateLimiter
import numpy as np  # Import NumPy

from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim  # Replace with your geocoding service if different
import time

# os.chdirc('')

In [586]:
# Adjust the display settings to see the full content of the column
pd.set_option('display.max_colwidth', None)

# Set display option to show all rows
pd.set_option('display.max_rows', None)

# print columns function
def col_printing(df):
    for i, col in enumerate(df):
        print(i, col)

# Loading in the Census Tract shapefiles

In [ ]:
# path to shapefile
shapefile_census = 'C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/WI_Tract/tl_2023_55_tract.shp'

# load the shapefile into geodataframe
tract_gdf = gpd.read_file(shapefile_census)

tract_gdf.to_csv('tract_gdf.csv', index = False)

# rename the GEOIDFQ to ID
tract_gdf = tract_gdf.rename(columns = {'GEOIDFQ' : 'ID'})

# start by reprojecting to EPSG: 26916, this allows us to calculate the area
tract_gdf = tract_gdf.to_crs(epsg=26916) 

# Add a new column for the area in square meters
tract_gdf['tract_area_sq_m'] = tract_gdf.geometry.area

# Loading in the Neighborhood shapefiles

In [ ]:
# path to shapefile
shapefile_neigh = 'C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/DCDNeighborhoods.shp'

# load the shapefile into geodataframe
neighbor_gdf = gpd.read_file(shapefile_neigh)

# display the first few rows of dataframe
print(neighbor_gdf.head())

# start by reprojecting to EPSG: 3857, this allows us to calculate the area
neighbor_gdf = neighbor_gdf.to_crs(epsg=26916)  
# Add a new column for the area in square meters
neighbor_gdf['neigh_area_sq_m'] = neighbor_gdf.geometry.area

neighbor_gdf.to_csv('neighbor_gdf.csv', index = False)

   OBJECTID    Neighborho  SYMBOL    Neighbor_1    SHAPE_STAr    SHAPE_STLe  \
0         3     NEW COELN       1     New Coeln  5.723546e+06   9615.009330   
1         4    SOUTHPOINT       4    Southpoint  1.073536e+07  14415.135317   
2         5  TOWN OF LAKE       2  Town Of Lake  2.737191e+07  25967.630731   
3         6    TIPPECANOE       4    Tippecanoe  2.645802e+07  33503.757162   
4         7    MORGANDALE       2    Morgandale  2.952580e+07  22330.698849   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

# Load in Demographic Details
S2301 - population, labor, unemployment, employment rate
S0101 - age x gender
B03002 - not hispanic
B02001 - race

## 2023 files
merged_23_df is the merged file for all of the 2023 data

In [ ]:
# read the files for 2023
df_S2301_emp = pd.read_csv("C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/ACSDT5Y2023.S2301-Transposed.csv")
df_S0101_age = pd.read_csv("C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/ACSDT5Y2023.S0101-Transposed.csv")
df_B03002_nothis = pd.read_csv("C:/blah/blah/Documents/Data/Misc/dcdneighborhoods/ACSDT5Y2023.B03002-Data.csv")
df_B02001_race = pd.read_csv("C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/ACSDT5Y2023.B02001-Data.csv")

# rename totalpop in df_S0101_age to totalpop_agedf
df_S0101_age = df_S0101_age.rename(columns = {'totalpop' : 'totalpop_agedf'})

# rename totalpop in df_B02001_race to totalpop_racedf
df_B02001_race = df_B02001_race.rename(columns = {'totalpop' : 'totalpop_racedf'})

merge1_df = pd.merge(df_B02001_race, df_S0101_age, on = 'ID', how = 'inner')
merged_23_df = pd.merge(merge1_df, df_B03002_nothis, on = 'ID', how = 'inner')
merged_23_df = pd.merge(merged_23_df, df_S2301_emp, on = 'ID', how = 'inner')

## 2022 files (acs5yr_2022_merged.csv)

In [ ]:
merged_22_df = pd.read_csv("C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/acs5yr_2022_merged.csv")

# rename id to ID
merged_22_df = merged_22_df.rename(columns = {'id' : 'ID', 'totalpop' : 'totalpop_2022'})

### Combine 2022 and 2023 dataframes together

In [591]:
# Merge DataFrames on the 'ID' column
combined_df = pd.merge(merged_22_df, merged_23_df, on='ID', suffixes=('_2022', '_2023'))

# totalpop_agedf and totalpop_racedf are the same so we drop totalpop_racedf and rename totalpop_agedf into totalpop2023
combined_df = combined_df.drop(columns=['totalpop_racedf'])
combined_df = combined_df.rename(columns = {'totalpop_agedf' : 'totalpop_2023'})

combined_df.to_csv('combined_nocrime.csv', index = False)

### Calculating the Average Scores for Demographic Details 2022 and 2023

In [592]:
# Identify columns ending with '_2022' and '_2023'
columns_2022 = [col for col in combined_df.columns if col.endswith('_2022')]
columns_2023 = [col for col in combined_df.columns if col.endswith('_2023')]

# Ensure the same base columns exist in both years
common_bases = [col.replace('_2022', '') for col in columns_2022 if col.replace('_2022', '_2023') in columns_2023]

for base in common_bases:
    col_2022 = f'{base}_2022'
    col_2023 = f'{base}_2023'

    # Get the data types of the actual columns from the DataFrame
    column_types_2022 = combined_df[col_2022].dtype
    column_types_2023 = combined_df[col_2023].dtype

    print(f"Data type of {col_2022}: {column_types_2022}")
    print(f"Data type of {col_2023}: {column_types_2023}")

Data type of totalpop_2022: float64
Data type of totalpop_2023: int64
Data type of white_2022: float64
Data type of white_2023: object
Data type of black_2022: float64
Data type of black_2023: object
Data type of ai_an_2022: float64
Data type of ai_an_2023: int64
Data type of asian_2022: float64
Data type of asian_2023: int64
Data type of nh_opi_2022: float64
Data type of nh_opi_2023: int64
Data type of otherrace_2022: float64
Data type of otherrace_2023: object
Data type of two_or_more_race_2022: float64
Data type of two_or_more_race_2023: object
Data type of total_under5_2022: float64
Data type of total_under5_2023: int64
Data type of total_5to9_2022: float64
Data type of total_5to9_2023: int64
Data type of total10to14_2022: float64
Data type of total10to14_2023: int64
Data type of total_15to19_2022: float64
Data type of total_15to19_2023: int64
Data type of total_20to24_2022: float64
Data type of total_20to24_2023: int64
Data type of total_25to29_2022: float64
Data type of total_25t

In [593]:
# some columns are not in numeric format and here, we are converting them to numeric
colstoconvert = ['white_2023', 'black_2023', 'otherrace_2023', 
                 'two_or_more_race_2023', 'total_nothispanic_2023',
                 'white_not_hisp_2023', 'black_not_hisp_2023', 'total_hispanic_2023',
                 'hispanic_white_2023', 'hispanic_otherrace_2023',
                 'hispanic_two_or_more_race_2023'
                 ]

# loop through each column in the list
for col in colstoconvert:
    # remove commas and convert to numeric
    combined_df[col] = combined_df[col].str.replace(',', '', regex=True)
    combined_df[col] = pd.to_numeric(combined_df[col], errors='coerce')

In [594]:
def calculate_yearly_averages(df, base_columns):
    """
    Calculate averages between 2022 and 2023 values and add them to the original DataFrame.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame containing *_2022 and *_2023 columns
    base_columns (list): List of base column names without year suffixes
    
    Returns:
    pandas.DataFrame: Original DataFrame with new averaged columns added
    """
    # Create a copy of the original DataFrame to avoid modifying it directly
    result_df = df.copy()
    
    for col in base_columns:
        col_2022 = f"{col}_2022"
        col_2023 = f"{col}_2023"
        
        # Calculate average and store in new column
        # The new column will be named {col}_avg
        result_df[f"{col}_avg"] = df[[col_2022, col_2023]].mean(axis=1)
    
    return result_df

# Use the function
combined_df = calculate_yearly_averages(combined_df, common_bases)

In [595]:
# rename combined_df['tractfips'] to GEOID
combined_df = combined_df.rename(columns = {'tractfips' : 'GEOID'})

combined_df.to_csv("combined_df.csv", index = False)

### Add the geometry columns of our census tract file

In [ ]:
# CENSUS TRACT SHAPEFILE
shapefile_pathtract = 'C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/WI_Tract/tl_2023_55_tract.shp'
tract_gdf = gpd.read_file(shapefile_pathtract)
tract_gdf.to_csv('tract_gdf.csv', index = False)
print(tract_gdf.crs)

# NEIGHBORHOOD TRACT SHAPEFILE
shapefile_pathneigh = 'C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/DCDNeighborhoods.shp'
neighborhood_gdf = gpd.read_file(shapefile_pathneigh)
neighborhood_gdf.to_csv('neighborhood_gdf.csv', index = False)
print(tract_gdf.crs)

neighborhood_gdf = neighborhood_gdf.to_crs("EPSG:26916")
tract_gdf = tract_gdf.to_crs("EPSG:26916")

EPSG:4269
EPSG:4269


#### Merging tract_gdf with combined_df

In [597]:
tract_gdf['GEOID'] = tract_gdf['GEOID'].astype('int64')

combined_acs5yr_map_df = pd.merge(combined_df, tract_gdf, on = 'GEOID', how = 'inner')

acs5yr_2022_2023_avgonly = combined_acs5yr_map_df.copy()

acs5yr_2022_2023_avgonly.to_csv("acs5yr_2022_2023_avgonly.csv", index = False)

##### Retain specific columns for acs5yr_2022_2023_avgonly (remove the _2022 and _2023 columns) and calling this new dataframe, map_acs5yr

In [598]:
# specify columns to keep
column_indices = list(range(0, 3)) + list(range(92, 98)) + list(range(191, 298))

# Subset the DataFrame to keep only the specified columns
map_acs5yr = acs5yr_2022_2023_avgonly.iloc[:, column_indices]

map_acs5yr.to_csv('map_acs5yr.csv', index = False)

## Merging our map_acs5yr file with neighborhood shapefile (neighborhood_gdf) to find tract and neighborhood intersections

In [599]:
# Create GeoDataFrame with proper CRS
map_acs5yr = gpd.GeoDataFrame(
    map_acs5yr,
    geometry='geometry',  # Ensure you're using the correct column for geometry
    crs="EPSG:26916"  # Set the CRS to EPSG:26916 (NAD83 / Wisconsin South)
)

neighborhood_gdf = neighborhood_gdf.to_crs("EPSG:26916")

In [600]:
# Perform spatial intersection to find where tracts and neighborhoods overlap
intersection = gpd.overlay(map_acs5yr, neighborhood_gdf, how='intersection')

# Calculate the area of intersection for each tract-neighborhood pair
intersection['tractneigh_intersection_area'] = intersection.geometry.area

# Group by neighborhood to see which tracts are in each neighborhood
neighborhood_tracts = intersection.groupby('Neighborho').agg({
    'GEOID': list,  # List of tract IDs
    'NAME': list,   # List of tract names
    'tractneigh_intersection_area': list  # Areas of intersection
}).reset_index()

neighborhood_tracts.to_csv('neighborhood_tracts.csv', index = False)

In [601]:
# read in neighborhood_tracts
neighborhood_tracts = pd.read_csv('neighborhood_tracts.csv')

# Figuring out the Neighborhood Demographic Details

In [602]:
# clean up the three columns that are in percentages
# Replace '-' with NaN
for col in ['Labor Force Participation Rate', 'Employment/Population Ratio', 'Unemployment rate']:
    map_acs5yr[col] = map_acs5yr[col].replace('-', np.nan)

# Remove '%' and convert columns to numeric
for col in ['Labor Force Participation Rate', 'Employment/Population Ratio', 'Unemployment rate']:
    map_acs5yr[col] = map_acs5yr[col].str.rstrip('%').astype(float)

    # fill in NaN for empty elements
    map_acs5yr = map_acs5yr.replace('', np.nan)

avg_cols = ['Labor Force Participation Rate', 'Employment/Population Ratio', 'Unemployment rate']
# Replace '-' with NaN and then convert to float
map_acs5yr[avg_cols] = map_acs5yr[avg_cols].replace('-', np.nan).replace('%', '', regex=True).astype(float)

In [603]:
def estimate_neighborhood_demographics(tract_gdf, neighborhood_gdf, demographic_columns, rate_columns):
    """
    Estimate neighborhood demographics based on overlapping census tracts.
    
    Parameters:
    tract_gdf: GeoDataFrame with census tract data including demographics and geometry
    neighborhood_gdf: GeoDataFrame with neighborhood boundaries
    demographic_columns: List of demographic column names to estimate (raw demographic data)
    rate_columns: List of rate column names to estimate (percentage columns like Labor Force Participation Rate, etc.)
    
    Returns:
    GeoDataFrame with estimated neighborhood demographics
    """
    # Create copies to avoid modifying original data
    tracts = tract_gdf.copy()
    neighborhoods = neighborhood_gdf.copy()
    
    # Calculate tract areas for later proportion calculations
    tracts['tract_area'] = tracts.geometry.area
    
    # Perform spatial intersection
    intersections = gpd.overlay(tracts, neighborhoods, how = 'intersection')
    
    # Calculate the area of each intersection
    intersections['overlap_area'] = intersections.geometry.area
    
    # Calculate what proportion of each tract falls in each neighborhood
    intersections['tract_proportion'] = intersections['overlap_area'] / intersections['tract_area']
    
    # Initialize a dictionary to store results
    demographic_estimates = {}
    
    # Calculate weighted demographics for each neighborhood
    for col in demographic_columns:
        # Multiply demographic values by the proportion of tract area
        intersections[f'{col}_weighted'] = intersections[col] * intersections['tract_proportion']
        
        # Sum up the weighted values for each neighborhood
        demographic_estimates[col] = intersections.groupby('Neighborho')[f'{col}_weighted'].sum()
    
    # Handle rate columns differently (calculate average of rates within each neighborhood)
    for rate_col in rate_columns:
        # Take the weighted average of the rate columns (not the raw values)
        intersections[f'{rate_col}_avg'] = intersections[rate_col]
        
        # Calculate the average rate for each neighborhood (sum of rates / number of tracts in that neighborhood)
        demographic_estimates[rate_col] = intersections.groupby('Neighborho')[f'{rate_col}_avg'].mean()
    
    # Create results DataFrame
    results = pd.DataFrame(demographic_estimates)
    
    # Merge results back with original neighborhood data
    final_results = neighborhoods.merge(results, left_on='Neighborho', right_index=True)
    
    return final_results

def verify_estimates(original_tracts, estimated_neighborhoods, demographic_columns, rate_columns):
    """
    Verify that total populations roughly match between tracts and neighborhoods, 
    and handle verification for columns with percentage values by checking averages.
    """
    print("Verification of estimates:")
    print("-" * 50)
    
    # Verify demographic columns (non-percentage columns)
    for col in demographic_columns:
        tract_total = original_tracts[col].sum()
        neighborhood_total = estimated_neighborhoods[col].sum()
        difference_pct = ((neighborhood_total - tract_total) / tract_total) * 100
        
        print(f"\n{col}:")
        print(f"Total from tracts: {tract_total:,.0f}")
        print(f"Total from neighborhoods: {neighborhood_total:,.0f}")
        print(f"Difference: {difference_pct:,.2f}%")
    
    # Verify rate columns (percentage columns like Labor Force Participation Rate, etc.)
    for rate_col in rate_columns:
        tract_avg = original_tracts[rate_col].mean()
        neighborhood_avg = estimated_neighborhoods[rate_col].mean()
        difference_pct = ((neighborhood_avg - tract_avg) / tract_avg) * 100
        
        print(f"\n{rate_col}:")
        print(f"Average from tracts: {tract_avg:,.2f}%")
        print(f"Average from neighborhoods: {neighborhood_avg:,.2f}%")
        print(f"Difference: {difference_pct:,.2f}%")


In [604]:
demographic_cols = ['Population', 'totalpop_avg', 'white_avg', 'black_avg', 'asian_avg', 'ai_an_avg', 'nh_opi_avg', 'otherrace_avg', 
    'two_or_more_race_avg', 'total_under5_avg', 'total_5to9_avg', 'total10to14_avg', 
    'total_15to19_avg', 'total_20to24_avg', 'total_25to29_avg', 'total_30to34_avg', 
    'total_35to39_avg', 'total_40to44_avg', 'total_45to49_avg', 'total_50to54_avg', 
    'total_55to59_avg', 'total_60to64_avg', 'total_65to69_avg', 'total_70to74_avg', 
    'total_75to79_avg', 'total_80to84_avg', 'total_85andolder_avg', 'total_under18_avg', 
    'total_16andover_avg', 'total_over18_avg', 'totalmale_avg', 'male_under5_avg', 
    'male_5to9_avg', 'male_10to14_avg', 'male_15to19_avg', 'male_20to24_avg', 'male_25to29_avg',
    'male_30to34_avg', 'male_35to39_avg', 'male_40to44_avg', 'male_45to49_avg', 'male_50to54_avg',
    'male_55to59_avg', 'male_60to64_avg', 'male_65to69_avg', 'male_70to74_avg', 'male_75to79_avg',
    'male_80to84_avg', 'male_85andover_avg', 'male_under18_avg', 'male_16andover_avg', 'male_over18_avg',
    'totalfemale_avg', 'female_under5_avg', 'female_5to9_avg', 'female_10to14_avg', 'female_15to19_avg',
    'female_20to24_avg', 'female_25to29_avg', 'female_30to34_avg', 'female_35to39_avg', 'female_40to44_avg',
    'female_45to49_avg', 'female_50to54_avg', 'female_55to59_avg', 'female_60to64_avg', 'female_65to69_avg',
    'female_70to74_avg', 'female_75to79_avg', 'female_80to84_avg', 'female_85andover_avg', 'female_under18_avg',
    'female_16andover_avg', 'female_over18_avg', 'total_nothispanic_avg', 'white_not_hisp_avg', 'black_not_hisp_avg',
    'ai_an_not_hisp_avg', 'asian_not_hisp_avg', 'nh_opi_not_hisp_avg', 'otherrace_not_hisp_avg', 'two_or_more_race_not_hisp_avg',
    'total_hispanic_avg', 'hispanic_white_avg', 'hispanic_black_avg', 'hispanic_ai_an_avg', 'hispanic_asian_avg',
    'hispanic_nh_opi_avg', 'hispanic_otherrace_avg', 'hispanic_two_or_more_race_avg']

rate_cols = ['Labor Force Participation Rate', 'Employment/Population Ratio', 'Unemployment rate']

# Calculate estimates
neighborhood_estimates = estimate_neighborhood_demographics(
    map_acs5yr,    # your tract GeoDataFrame
    neighborhood_gdf,  # your neighborhood GeoDataFrame
    demographic_cols, rate_cols   # list of columns to estimate
)

# Verifying estimates
verified_numbers = verify_estimates(map_acs5yr, neighborhood_estimates, demographic_cols, rate_cols) 

Verification of estimates:
--------------------------------------------------

Population:
Total from tracts: 692,047
Total from neighborhoods: 551,126
Difference: -20.36%

totalpop_avg:
Total from tracts: 884,260
Total from neighborhoods: 713,014
Difference: -19.37%

white_avg:
Total from tracts: 384,864
Total from neighborhoods: 260,910
Difference: -32.21%

black_avg:
Total from tracts: 306,655
Total from neighborhoods: 290,155
Difference: -5.38%

asian_avg:
Total from tracts: 39,757
Total from neighborhoods: 31,292
Difference: -21.29%

ai_an_avg:
Total from tracts: 5,400
Total from neighborhoods: 4,489
Difference: -16.88%

nh_opi_avg:
Total from tracts: 141
Total from neighborhoods: 110
Difference: -21.94%

otherrace_avg:
Total from tracts: 56,418
Total from neighborhoods: 48,492
Difference: -14.05%

two_or_more_race_avg:
Total from tracts: 91,026
Total from neighborhoods: 77,566
Difference: -14.79%

total_under5_avg:
Total from tracts: 59,192
Total from neighborhoods: 49,536
Differ

### neighborhood_estimates contains demographic details by each neighborhood

In [605]:
# Rename the Neighborho column to Neighborhood
neighborhood_estimates = neighborhood_estimates.rename(columns = {'Neighborho' : 'Neighborhood'})

# Lower case the Neighborhood values
neighborhood_estimates['Neighborhood'] = neighborhood_estimates['Neighborhood'].fillna('').apply(str)
neighborhood_estimates['Neighborhood'] = neighborhood_estimates['Neighborhood'].str.strip().str.lower()

# Export to CSV
neighborhood_estimates.to_csv('neighborhood_estimates.csv', index = False)

# Bringing in the Victims and Suspects dataframes

In [ ]:
# read the suspect and victims dataframe
df_victims = pd.read_csv("C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/2022-2023 - Part I Violent Crime and Part II Crime Against Person - Victim.csv")
df_suspects = pd.read_csv("C:/Users/blah/blah/Documents/Data/Misc/dcdneighborhoods/2022-2023 - Part I Violent Crime and Part II Crime Against Person - Suspect.csv")

# Cleaning the Neighborhood column for df_victims
# Replace 'nan' (string) with np.nan
df_victims['Neighborhood'] = df_victims['Neighborhood'].replace('nan', np.nan)
# Convert the 'Neighborhood' column to string values and then apply string methods
df_victims['Neighborhood'] = df_victims['Neighborhood'].fillna('').apply(str)
df_victims['Neighborhood'] = df_victims['Neighborhood'].str.strip().str.lower()

# Drop rows where 'Neighborhood' is either NaN or an empty string
df_victims = df_victims[df_victims['Neighborhood'].str.strip() != '']

# Cleaning the Neighborhood column for df_suspects
# Replace 'nan' (string) with np.nan
df_suspects['Neighborhood'] = df_suspects['Neighborhood'].replace('nan', np.nan)
# Convert the 'Neighborhood' column to string values and then apply string methods
df_suspects['Neighborhood'] = df_suspects['Neighborhood'].fillna('').apply(str)
df_suspects['Neighborhood'] = df_suspects['Neighborhood'].str.strip().str.lower()

# Drop rows where 'Neighborhood' is either NaN or an empty string
df_suspects = df_suspects[df_suspects['Neighborhood'].str.strip() != '']

# Renaming the race-ethnicity column
race_dict = {"Black / African American" : "black",
             "Hispanic / Latino" : "hispanic",
             "Native Hawaiian / Other Pacific Islander" : "NH_PI",
             "Asian" : "Asian",
             "White" : "white",
             "American Indian / Alaskan Native" : "AI_AN",
             "Unknown / Other" : "other"}

df_victims['Race-Ethnicity'] = df_victims['Race-Ethnicity'].replace(race_dict)
df_suspects['Race-Ethnicity'] = df_suspects['Race-Ethnicity'].replace(race_dict)

# Renaming the offenses
offense_dict = {"Aggravated Assault" : 'total_aggravated_assault',
                                                  "Crime Against Person" : "total_partIIperson",
                                                  'Homicide' : 'total_homicide',
                                                  'Human Trafficking' : 'total_human_trafficking',
                                                  'Rape' : 'total_rape',
                                                  'Robbery' : 'total_robbery'}

df_suspects['Offense Type'] = df_suspects['Offense Type'].replace(offense_dict)
df_victims['Offense Type'] = df_victims['Offense Type'].replace(offense_dict)

In [607]:
# victim counts
pivot_vic_offrace = df_victims.groupby(['Neighborhood', 'Race-Ethnicity', 'Offense Type']).size().unstack(['Race-Ethnicity', 'Offense Type'], fill_value=0)
df_victims['Race_Offense'] = df_victims['Race-Ethnicity'] + '_' + df_victims['Offense Type']
pivot_vic_offrace = df_victims.groupby(['Neighborhood', 'Race_Offense']).size().unstack(fill_value=0)

pivot_vic_offtype = df_victims.groupby(['Neighborhood', 'Race_Offense']).size().unstack(fill_value=0)

# Create a combined column for Race-Ethnicity and Offense Category
df_victims['Race_OffenseCat'] = df_victims['Race-Ethnicity'] + '_' + df_victims['Offense Category']
pivot_vic_offcat_race = df_victims.groupby(['Neighborhood', 'Race_OffenseCat']).size().unstack(fill_value=0)

df_victims_new = pd.merge(pivot_vic_offcat_race, pivot_vic_offrace, left_index=True, right_index=True)
# Reset index to make 'Neighborhood' a column again
df_victims_new = df_victims_new.reset_index()

df_victims_new = pd.merge(df_victims_new, pivot_vic_offtype, on = 'Neighborhood', how = 'outer')
# Reset index to make 'Neighborhood' a column again
df_victims_new = df_victims_new.reset_index()

# FOR df_suspects (Offense in general)
# Group by 'Neighborhood' and 'Offense Type' to count occurrences
pivot_off_type = df_suspects.groupby(['Neighborhood', 'Offense Type']).size().unstack(fill_value=0)

# Create a new column combining 'Race-Ethnicity' and 'Offense Type'
df_suspects['Race_Offense'] = df_suspects['Race-Ethnicity'] + '_' + df_suspects['Offense Type']

# Correct aggregation: Directly group and unstack instead of using transform('size')
pivot_off_race = df_suspects.groupby(['Neighborhood', 'Race_Offense']).size().unstack(fill_value=0)

# FOR df_suspects (Part I offense)
# Create a new column combining 'Race-Ethnicity' and 'Offense Category'
df_suspects['Race_OffenseCat'] = df_suspects['Race-Ethnicity'] + '_' + df_suspects['Offense Category']

pivot_offcat_race = df_suspects.groupby(['Neighborhood', 'Race_OffenseCat']).size().unstack(fill_value=0)

# Merge both pivot tables on Neighborhood
df_suspects_new = pd.merge(pivot_offcat_race, pivot_off_race, 
                           left_index=True, right_index=True, how='outer')

# Reset index to make 'Neighborhood' a column again
df_suspects_new = df_suspects_new.reset_index()

# Merge with Offense Type pivot table
df_suspects_new = pd.merge(df_suspects_new, pivot_off_type, on='Neighborhood', how='outer')


# COMBINE VICTIMS AND SUSPECTS DATA INTO ONE DATAFRAME
combined_crime_df = pd.merge(df_victims_new, df_suspects_new, on = 'Neighborhood', suffixes = ('_vic', '_sus'))
# Export to CSV
combined_crime_df.to_csv('combined_crime_df.csv', index = False)


##### Merge neighborhood_estimates with victims and suspects data

In [ ]:
# read in neighborhood_estimates
neighborhood_estimates = pd.read_csv("C:/Users/blah/blah/Documents/Data/WI_RAD/neighborhood_estimates.csv")

In [609]:
neighborhood_vicsus_est = pd.merge(neighborhood_estimates, combined_crime_df, on = 'Neighborhood', how = 'inner')
neighborhood_vicsus_est.to_csv('neighborhood_vicsus_est.csv', index = False)

# Bringing in the Pedestrian Dataframe

In [ ]:
df_ped = pd.read_stata("C:/Users/blah/blah/Documents/Data/WI_RAD/persons_fionly_geolocated_foranalysis_2022-2023.dta")

# Rename the neighborhood column to Neighborhood
df_ped = df_ped.rename(columns = {'neighborhood':'Neighborhood'})
# Replace 'nan' (string) with np.nan
df_ped['Neighborhood'] = df_ped['Neighborhood'].replace('nan', np.nan)
# Convert the 'Neighborhood' column to string values and then apply string methods
df_ped['Neighborhood'] = df_ped['Neighborhood'].fillna('').apply(str)
df_ped['Neighborhood'] = df_ped['Neighborhood'].str.strip().str.lower()
# Drop rows where 'Neighborhood' is either NaN or an empty string
df_ped = df_ped[df_ped['Neighborhood'].str.strip() != '']
df_ped = df_ped[df_ped['Neighborhood'] != '1_could not locate']

In [611]:
## Figuring out the count of pedestrian encounters in each neighborhood
PedNeigh_counts = df_ped.groupby('Neighborhood').size().reset_index(name='ped_count')

# Bringing in the Vehicles/Encounter Dataframe

In [ ]:
df_enc = pd.read_stata("C:/Users/blah/blah/Documents/Data/WI_RAD/ts_encounters_analysis_updated key variables_2022-2023.dta")

# Rename the neighborhood column to Neighborhood
df_enc = df_enc.rename(columns = {'neighborhood':'Neighborhood'})
# Replace 'nan' (string) with np.nan
df_enc['Neighborhood'] = df_enc['Neighborhood'].replace('nan', np.nan)
# Convert the 'Neighborhood' column to string values and then apply string methods
df_enc['Neighborhood'] = df_enc['Neighborhood'].fillna('').apply(str)
df_enc['Neighborhood'] = df_enc['Neighborhood'].str.strip().str.lower()
# Drop rows where 'Neighborhood' is either NaN or an empty string
df_enc = df_enc[df_enc['Neighborhood'].str.strip() != '']
df_enc = df_enc[df_enc['Neighborhood'] != '1_could not locate']

In [613]:
## Figuring out the count of vehicle encounters in each neighborhood
EncNeigh_counts = df_enc.groupby('Neighborhood').size().reset_index(name='enc_count')

##### Creating a dataframe with neighborhood estimates, victims and suspects, and pedestrian and vehicle counts

In [614]:
neighborhood_vicsus_pedenc_est = pd.merge(neighborhood_vicsus_est, EncNeigh_counts, on = 'Neighborhood', how = 'inner')
neighborhood_vicsus_pedenc_est = pd.merge(neighborhood_vicsus_pedenc_est, PedNeigh_counts, on = 'Neighborhood', how = 'inner')
neighborhood_vicsus_pedenc_est.to_csv('neighborhood_vicsus_pedenc_est.csv', index = False)

# New Dataframe with Pedestrians as the Unit of Analysis (df_ped)

### Start with creating a count of victims by race in the df_victims dataframe

#### Count of victims (Part 1 and 2 separated)

In [615]:
# group by Neighborhood and offense Category, and count occurrences of Race-Ethnicity
offcat_counts = df_victims.groupby(['Neighborhood', 'Race-Ethnicity', 'Offense Category']).size().reset_index(name='Count')

# create a combined column for Race-Ethnicity and offense Category
offcat_counts['Race_Offense'] = offcat_counts['Race-Ethnicity'] + '_' + offcat_counts['Offense Category']

# pivot the table
race_viccat_neigh_pivot = offcat_counts.pivot(index='Neighborhood', columns='Race_Offense', values='Count')
race_viccat_neigh_pivot = race_viccat_neigh_pivot.fillna(0)

# rename the columns
race_viccat_neigh_pivot = race_viccat_neigh_pivot.rename(columns = {'AI_AN_Part I' : 'AI_AN_offcat1_viccnt',
                                                                    'AI_AN_Part II' : 'AI_AN_offcat2_viccnt',
                                                                    'Asian_Part I' : 'Asian_offcat1_viccnt', 
                                                                    'Asian_Part II' : 'Asian_offcat2_viccnt',
                                                                    'NH_PI_Part I' : 'NH_PI_offcat1_viccnt',
                                                                    'NH_PI_Part II' : 'NH_PI_offcat2_viccnt',
                                                                    'black_Part I' : 'black_offcat1_viccnt',
                                                                    'black_Part II' : 'black_offcat2_viccnt',
                                                                    'hispanic_Part I' : 'hispanic_offcat1_viccnt',
                                                                    'hispanic_Part II' : 'hispanic_offcat2_viccnt',
                                                                    'other_Part I' : 'other_offcat1_viccnt',
                                                                    'other_Part II' : 'other_offcat2_viccnt',
                                                                    'white_Part I' : 'white_offcat1_viccnt',
                                                                    'white_Part II' : 'white_offcat2_viccnt'})

print(race_viccat_neigh_pivot)
race_viccat_neigh_pivot = race_viccat_neigh_pivot.reset_index()

Race_Offense                AI_AN_offcat1_viccnt  AI_AN_offcat2_viccnt  \
Neighborhood                                                             
alcott park                                  0.0                   0.0   
alverno                                      0.0                   0.0   
amani                                        0.0                   2.0   
arlington gardens                            0.0                   0.0   
arlington heights                            1.0                   1.0   
avenues west                                 0.0                   2.0   
baran park                                   0.0                   0.0   
bay view                                     4.0                   3.0   
bluemound heights                            0.0                   0.0   
borchert field                               3.0                   0.0   
bradley estates                              0.0                   0.0   
brewer's hill                         

In [616]:
race_viccat_neigh_pivot.to_csv('race_viccat_neigh_pivot.csv', index = False)

#### Count of victims (Part 1 and 2 COMBINED)

In [617]:
# race of victims count by neighborhoods (part I and part II crimes are combined)
race_vic_neigh = df_victims.groupby(['Neighborhood', 'Race-Ethnicity']).size().unstack(fill_value=0).reset_index()

race_vic_neigh = race_vic_neigh.rename(columns = {'AI_AN' : 'AI_AN_genviccnt',
                                                  'Asian': 'Asian_genviccnt',
                                                  'NH_PI': 'NH_PI_genviccnt',
                                                  'black':'black_genviccnt',
                                                  'white' : 'white_genviccnt',
                                                  'other' : 'other_genviccnt',
                                                  'hispanic':'hispanic_genviccnt'})

### Creating a count of pedestrian stops by race in the df_ped dataframe

In [618]:
# Use cat.rename_categories to rename 'missing' to NaN
df_ped['racecat'] = df_ped['racecat'].cat.remove_categories('missing')

race_count_neigh = df_ped.pivot_table(index = 'Neighborhood', columns = 'racecat', aggfunc = 'size', fill_value = 0, observed = True)

race_count_neigh = race_count_neigh.rename(columns = {'white' : 'white_peds', 'black' : 'black_peds',
                                                      'latinx' : 'hispanic_peds', 
                                                      'other' : 'other_peds'})

# Reset the index so that 'neighborhood' becomes a column
race_count_neigh = race_count_neigh.reset_index()

##### Merge the race_count_neigh and race_vic_neigh dataframe together

In [619]:
viccombined_ped_count = pd.merge(race_vic_neigh, race_count_neigh, on = 'Neighborhood', how = 'inner')

##### Merge the race_count_neigh and race_viccat_neigh_pivot dataframe together (for the separated offense category dataframe)

In [620]:
vic_offcat1and2_ped_count = pd.merge(race_viccat_neigh_pivot, race_count_neigh, on = 'Neighborhood', how = 'inner')

In [621]:
vic_offcat1and2_ped_count.to_csv('vic_offcat1and2_ped_count.csv', index = False)

In [622]:
col_printing(vic_offcat1and2_ped_count)

0 Neighborhood
1 AI_AN_offcat1_viccnt
2 AI_AN_offcat2_viccnt
3 Asian_offcat1_viccnt
4 Asian_offcat2_viccnt
5 NH_PI_offcat1_viccnt
6 NH_PI_offcat2_viccnt
7 black_offcat1_viccnt
8 black_offcat2_viccnt
9 hispanic_offcat1_viccnt
10 hispanic_offcat2_viccnt
11 other_offcat1_viccnt
12 other_offcat2_viccnt
13 white_offcat1_viccnt
14 white_offcat2_viccnt
15 white_peds
16 black_peds
17 hispanic_peds
18 other_peds


## Forming the pedestrian df with victim counts and pedestrian counts

#### COMBINED OFFENSES

In [623]:
# OFEENSES ARE COMBINED
df_ped_viccombined_count = pd.merge(df_ped, viccombined_ped_count, on = 'Neighborhood', how = 'inner')
df_ped_viccombined_count.to_csv("df_ped_viccombined_count.csv", index = False)

#### Seperated Offenses (Part 1 and 2)

In [624]:
## OFFENSES ARE SEPARATED
df_ped_vic1and2_count = pd.merge(df_ped, vic_offcat1and2_ped_count, on = 'Neighborhood', how = 'inner')
df_ped_vic1and2_count.to_csv("df_ped_vic1and2_count.csv", index = False)

## Forming the pedestrian df (unit of analysis) with victim counts, pedestrian counts, and neighborhood details

#### Combined offenses

In [625]:
df_ped_viccombined_count = pd.merge(df_ped_viccombined_count, neighborhood_estimates, on = 'Neighborhood', how = 'inner')
df_ped_viccombined_count.to_csv("df_ped_viccombined_count.csv", index = False)

#### Seperated offenses

In [626]:
df_ped_vic1and2_count = pd.merge(df_ped_vic1and2_count, neighborhood_estimates, on = 'Neighborhood', how = 'inner')
df_ped_vic1and2_count.to_csv("df_ped_vic1and2_count.csv", index = False)

### Tidying up the def_ped_viccombined_count and the df_ped_vic1and2_count dataframes  by removing unnecessary variables

#### Starting with the combined offense dataframe (df_ped_viccombined_count)

In [627]:
col_keep = list(range(0, 23)) + list(range(28, 55)) + list(range(668, 671)) + list(range(1613, 1619)) + list(range(1626, 1760)) 

df_ped_viccombined_count = df_ped_viccombined_count.iloc[:, col_keep]
df_ped_viccombined_count.to_csv('df_ped_viccombined_count.csv', index = False)

#### Seperated offense dataframe

In [628]:
col_keep = list(range(0, 23)) + list(range(28, 55)) + list(range(668, 671)) + list(range(1613, 1619)) + list(range(1626, 1767)) 

df_ped_vic1and2_count = df_ped_vic1and2_count.iloc[:, col_keep]
df_ped_vic1and2_count.to_csv('df_ped_vic1and2_count.csv', index = False)

### Adding More Variables into ped_vic_neigh_count
Variables like: 
- totalvictims - total_viccnt - (AI_AN_viccnt, Asian_viccnt, NH_PI_viccnt, black_viccnt, hispanic_viccnt, other_viccnt, white_viccnt)
- totalblackwhite_pedstops (white_peds + black_peds)
- black stop rate (black stops/black+white stops)
- white stop rate (white stops/black+white stops)
- population proportion of black individuals (black population/totalpopulation)
- population proportion of white individuals (white population/totalpopulation)
- black_ped_DI (black stop rate/population proportion of black individuals)
- white_ped_DI (white stop rate/population proportion of white individuals)

##### Calculating total victims (combined offenses)

In [629]:
victimgrps = ['AI_AN_genviccnt', 'Asian_genviccnt', 'NH_PI_genviccnt', 'black_genviccnt', 'hispanic_genviccnt', 'other_genviccnt', 'white_genviccnt']
race_vic_neigh['total_genviccnt'] = race_vic_neigh[victimgrps].sum(axis = 1)
race_vic_neigh.head()

Race-Ethnicity,Neighborhood,AI_AN_genviccnt,Asian_genviccnt,NH_PI_genviccnt,black_genviccnt,hispanic_genviccnt,other_genviccnt,white_genviccnt,total_genviccnt
0,alcott park,0,0,0,3,0,0,3,6
1,alverno,0,0,0,4,7,0,12,23
2,amani,2,0,0,262,6,0,10,280
3,arlington gardens,0,1,0,114,6,0,23,144
4,arlington heights,2,5,0,539,14,0,46,606


##### Calculating total victims (separated offenses)

In [630]:
victimgrps_cat1 = ['AI_AN_offcat1_viccnt', 'Asian_offcat1_viccnt', 'NH_PI_offcat1_viccnt', 'black_offcat1_viccnt', 'hispanic_offcat1_viccnt', 'other_offcat1_viccnt', 'white_offcat1_viccnt']
victimsgrps_cat2 = ['AI_AN_offcat2_viccnt', 'Asian_offcat2_viccnt', 'NH_PI_offcat2_viccnt', 'black_offcat2_viccnt', 'hispanic_offcat2_viccnt', 'other_offcat2_viccnt', 'white_offcat2_viccnt']
race_viccat_neigh_pivot['total_cat1viccnt'] = race_viccat_neigh_pivot[victimgrps_cat1].sum(axis = 1)
race_viccat_neigh_pivot['total_cat2viccnt'] = race_viccat_neigh_pivot[victimsgrps_cat2].sum(axis = 1)

In [631]:
race_viccat_neigh_pivot.head()

Race_Offense,Neighborhood,AI_AN_offcat1_viccnt,AI_AN_offcat2_viccnt,Asian_offcat1_viccnt,Asian_offcat2_viccnt,NH_PI_offcat1_viccnt,NH_PI_offcat2_viccnt,black_offcat1_viccnt,black_offcat2_viccnt,hispanic_offcat1_viccnt,hispanic_offcat2_viccnt,other_offcat1_viccnt,other_offcat2_viccnt,white_offcat1_viccnt,white_offcat2_viccnt,total_cat1viccnt,total_cat2viccnt
0,alcott park,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,3.0,1.0,5.0
1,alverno,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,4.0,3.0,0.0,0.0,4.0,8.0,9.0,14.0
2,amani,0.0,2.0,0.0,0.0,0.0,0.0,169.0,93.0,3.0,3.0,0.0,0.0,5.0,5.0,177.0,103.0
3,arlington gardens,0.0,0.0,0.0,1.0,0.0,0.0,58.0,56.0,3.0,3.0,0.0,0.0,12.0,11.0,73.0,71.0
4,arlington heights,1.0,1.0,5.0,0.0,0.0,0.0,318.0,221.0,9.0,5.0,0.0,0.0,27.0,19.0,360.0,246.0


##### Calculating black and white victimization rates (combined offenses dataframe)

In [632]:
race_vic_neigh['black_genvicrate'] = race_vic_neigh['black_genviccnt'] / race_vic_neigh['total_genviccnt']
race_vic_neigh['white_genvicrate'] = race_vic_neigh['white_genviccnt'] / race_vic_neigh['total_genviccnt']

# subset race_vic_neigh to a smaller df for merging later on
vic_combinedoff_info = race_vic_neigh[['black_genvicrate', 'white_genvicrate', 'total_genviccnt', 'Neighborhood']]

##### Calculating black and white victimization rates (separated offenses dataframe)

In [633]:
# Offense Part 1
race_viccat_neigh_pivot['black_offcat1_vicrate'] = race_viccat_neigh_pivot['black_offcat1_viccnt'] / race_viccat_neigh_pivot['total_cat1viccnt']
race_viccat_neigh_pivot['white_offcat1_vicrate'] = race_viccat_neigh_pivot['white_offcat1_viccnt'] / race_viccat_neigh_pivot['total_cat1viccnt']

# Offense Part 2
race_viccat_neigh_pivot['black_offcat2_vicrate'] = race_viccat_neigh_pivot['black_offcat2_viccnt'] / race_viccat_neigh_pivot['total_cat2viccnt']
race_viccat_neigh_pivot['white_offcat2_vicrate'] = race_viccat_neigh_pivot['white_offcat2_viccnt'] / race_viccat_neigh_pivot['total_cat2viccnt']


In [634]:
race_viccat_neigh_pivot = race_viccat_neigh_pivot.reset_index()
# subset race_vic_neigh to a smaller df for merging later on
vic_offcat1and2_info = race_viccat_neigh_pivot[['black_offcat1_vicrate', 'black_offcat2_vicrate', 'white_offcat1_vicrate', 'white_offcat2_vicrate', 'total_cat1viccnt', 'total_cat2viccnt', 'Neighborhood']]

##### Calculating stop rates for Black and White pedestrians (this can be used for both the combined offenses dataframe and the separated offenses dataframe)

In [635]:
# recode racecat so that missing = NA
racecat_missing = {'missing' : np.nan}
df_enc['racecat'] = df_enc['racecat'].replace(racecat_missing)
df_ped['racecat'] = df_ped['racecat'].replace(racecat_missing)

# Creating a pivot table that lets us see the counts of vehicle stops by race by neighborhood
encneighrace_pivot = df_enc.pivot_table(
    index='Neighborhood',   # Rows (neighborhoods)
    columns='racecat',      # Columns (races)
    aggfunc='size',         # Count occurrences
    fill_value=0            # Replace NaNs with 0s
)

encneighrace_pivot = encneighrace_pivot.reset_index()

# Creating a pivot table that lets us see the counts of pedestrian stops by race by neighborhood
pedneighrace_pivot = df_ped.pivot_table(
    index='Neighborhood',   # Rows (neighborhoods)
    columns='racecat',      # Columns (races)
    aggfunc='size',         # Count occurrences
    fill_value=0            # Replace NaNs with 0s
)

pedneighrace_pivot = pedneighrace_pivot.reset_index()

# merge pedneighrace_pivot into neighborhood_vicsus_est
neighborhood_vicsus_pedenc_final = pd.merge(neighborhood_vicsus_est, pedneighrace_pivot, on = 'Neighborhood', how = 'inner')

# rename the pedestrian columns
neighborhood_vicsus_pedenc_final = neighborhood_vicsus_pedenc_final.rename(columns = {'white' : 'white_pedstops',
                                                                                      'black' : 'black_pedstops',
                                                                                      'latinx' : 'latinx_pedstops',
                                                                                      'other' : 'other_pedstops'})

# merge encneighrace_pivot into neighborhood_vicsus_pedenc_final
neighborhood_vicsus_pedenc_final = pd.merge(neighborhood_vicsus_pedenc_final, encneighrace_pivot, on = 'Neighborhood', how = 'inner')

# rename the vehicle columns
neighborhood_vicsus_pedenc_final = neighborhood_vicsus_pedenc_final.rename(columns = {'white' : 'white_vehstops',
                                                                                      'black' : 'black_vehstops',
                                                                                      'latinx' : 'latinx_vehstops',
                                                                                      'other' : 'other_vehstops'})


C:\Users\michelle.iolow\AppData\Local\Temp\ipykernel_7520\2432590672.py:3: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  df_enc['racecat'] = df_enc['racecat'].replace(racecat_missing)
C:\Users\michelle.iolow\AppData\Local\Temp\ipykernel_7520\2432590672.py:7: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  encneighrace_pivot = df_enc.pivot_table(
C:\Users\michelle.iolow\AppData\Local\Temp\ipykernel_7520\2432590672.py:17: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current 

In [636]:
new_ped_df = neighborhood_vicsus_pedenc_final.copy()

pedgrps = ['white_pedstops', 'black_pedstops']
# creating a variable that has the total pedestrian stops for black and white people
new_ped_df['black_white_total_pedstops'] = new_ped_df[pedgrps].sum(axis = 1)

# calculating white and black stop rate
new_ped_df['white_ped_stoprate'] = new_ped_df['white_pedstops']/new_ped_df['black_white_total_pedstops']
new_ped_df['black_ped_stoprate'] = new_ped_df['black_pedstops']/new_ped_df['black_white_total_pedstops']

# subset this dataframe for merging later
new_ped_df = new_ped_df[['white_ped_stoprate', 'black_ped_stoprate', 'black_white_total_pedstops', 'Neighborhood']]

##### Calculating Population Proportion for Black and White folks by neighborhood

In [637]:
new_pop_df = neighborhood_vicsus_pedenc_final.copy()

popgrps = ['white_avg', 'black_avg', 'asian_avg', 'ai_an_avg', 'nh_opi_avg', 'otherrace_avg', 'two_or_more_race_avg']
# creating a variable that has the total population count for each neighborhood
new_pop_df['totalpopulation_count'] = new_pop_df[popgrps].sum(axis = 1)

# calculating population proportion for white and black individuals in different neighborhood
new_pop_df['blackpop_prop'] = new_pop_df['black_avg']/new_pop_df['totalpopulation_count']
new_pop_df['whitepop_prop'] = new_pop_df['white_avg']/new_pop_df['totalpopulation_count']

# subset this dataframe for merging later
new_pop_df = new_pop_df[['totalpopulation_count', 'blackpop_prop', 'whitepop_prop', 'Neighborhood']]

## Calculating the non-RAD Disparity Index for Pedestrian Stops
This is stop rate of a given group divided population proportion of that given group

##### For the combined offenses dataframe

In [638]:
disparity_ped_combinedoff_df = pd.merge(new_ped_df, viccombined_ped_count, on = 'Neighborhood', how = 'inner')
disparity_ped_combinedoff_df = pd.merge(disparity_ped_combinedoff_df, new_pop_df, on = 'Neighborhood', how = 'inner')

In [639]:
col_printing(disparity_ped_combinedoff_df)

0 white_ped_stoprate
1 black_ped_stoprate
2 black_white_total_pedstops
3 Neighborhood
4 AI_AN_genviccnt
5 Asian_genviccnt
6 NH_PI_genviccnt
7 black_genviccnt
8 hispanic_genviccnt
9 other_genviccnt
10 white_genviccnt
11 white_peds
12 black_peds
13 hispanic_peds
14 other_peds
15 totalpopulation_count
16 blackpop_prop
17 whitepop_prop


##### For the separated offenses dataframe

In [640]:
disparity_ped_cat1and2_df = pd.merge(new_ped_df, vic_offcat1and2_ped_count, on = 'Neighborhood', how = 'inner')
disparity_ped_cat1and2_df = pd.merge(disparity_ped_cat1and2_df, new_pop_df, on = 'Neighborhood', how = 'inner')

### Merging disparity information into pedestrian dataframes (df_ped_viccombined_count - combined offenses) (df_ped_vic1and2_count - separated offense)

In [641]:
Pedestrian_CombinedOff_RegDF = pd.merge(df_ped_viccombined_count, disparity_ped_combinedoff_df, on = 'Neighborhood', how = 'inner')

Pedestrian_SepOff_RegDF = pd.merge(df_ped_vic1and2_count, disparity_ped_cat1and2_df, on = 'Neighborhood', how = 'inner')

In [642]:
# Drop repeated columns in both dataframes
Pedestrian_CombinedOff_RegDF.drop(Pedestrian_CombinedOff_RegDF.columns[83:94], axis=1, inplace=True)

# Rename the _y variables 
Pedestrian_CombinedOff_RegDF = Pedestrian_CombinedOff_RegDF.rename(columns = {'AI_AN_genviccnt_y' : 'AI_AN_genviccnt', 
                                                                              'Asian_genviccnt_y' : 'Asian_genviccnt',
                                                                              'NH_PI_genviccnt_y' : 'NH_PI_genviccnt',
                                                                              'black_genviccnt_y' : 'black_genviccnt',
                                                                              'hispanic_genviccnt_y' : 'hispanic_genviccnt',
                                                                              'other_genviccnt_y' : 'other_genviccnt',
                                                                              'white_genviccnt_y' : 'white_genviccnt',
                                                                              'white_peds_y' : 'white_peds',
                                                                              'black_peds_y' : 'black_peds',
                                                                              'hispanic_peds_y' : 'hispanic_peds',
                                                                              'other_peds_y' : 'other_peds'})

Pedestrian_CombinedOff_RegDF.to_csv('Pedestrian_CombinedOff_RegDF.csv', index = False)

In [643]:
Pedestrian_CombinedOff_RegDF['white_ped_combinedoff_DI'] = Pedestrian_CombinedOff_RegDF['white_ped_stoprate'] / Pedestrian_CombinedOff_RegDF['whitepop_prop']
Pedestrian_CombinedOff_RegDF['black_ped_combinedoff_DI'] = Pedestrian_CombinedOff_RegDF['black_ped_stoprate'] / Pedestrian_CombinedOff_RegDF['blackpop_prop']

In [644]:
# Drop repeated columns in both dataframes
Pedestrian_SepOff_RegDF.drop(Pedestrian_SepOff_RegDF.columns[83:101], axis = 1, inplace=True)

# Rename the _y variables 
Pedestrian_SepOff_RegDF = Pedestrian_SepOff_RegDF.rename(columns = {'AI_AN_offcat1_viccnt_y' : 'AI_AN_offcat1_viccnt', 
                                                                              'Asian_offcat1_viccnt_y' : 'Asian_offcat1_viccnt',
                                                                              'NH_PI_offcat1_viccnt_y' : 'NH_PI_offcat1_viccnt',
                                                                              'black_offcat1_viccnt_y' : 'black_offcat1_viccnt',
                                                                              'hispanic_offcat1_viccnt_y' : 'hispanic_offcat1_viccnt',
                                                                              'other_offcat1_viccnt_y' : 'other_offcat1_viccnt',
                                                                              'white_offcat1_viccnt_y' : 'white_offcat1_viccnt',
                                                                              'white_peds_y' : 'white_peds',
                                                                              'black_peds_y' : 'black_peds',
                                                                              'hispanic_peds_y' : 'hispanic_peds',
                                                                              'other_peds_y' : 'other_peds',
                                                                              'AI_AN_offcat2_viccnt_y' : 'AI_AN_offcat2_viccnt', 
                                                                              'Asian_offcat2_viccnt_y' : 'Asian_offcat2_viccnt',
                                                                              'NH_PI_offcat2_viccnt_y' : 'NH_PI_offcat2_viccnt',
                                                                              'black_offcat2_viccnt_y' : 'black_offcat2_viccnt',
                                                                              'hispanic_offcat2_viccnt_y' : 'hispanic_offcat2_viccnt',
                                                                              'other_offcat2_viccnt_y' : 'other_offcat2_viccnt',
                                                                              'white_offcat2_viccnt_y' : 'white_offcat2_viccnt'})

Pedestrian_SepOff_RegDF.to_csv('Pedestrian_SepOff_RegDF.csv', index = False)


In [645]:
Pedestrian_SepOff_RegDF['white_ped_sepoff_DI'] = Pedestrian_SepOff_RegDF['white_ped_stoprate']/Pedestrian_SepOff_RegDF['whitepop_prop']
Pedestrian_SepOff_RegDF['black_ped_sepoff_DI'] = Pedestrian_SepOff_RegDF['black_ped_stoprate']/Pedestrian_SepOff_RegDF['blackpop_prop']

# Forming the vehicles regression dataframe

In [646]:
neighborhood_estimates = neighborhood_estimates.rename(columns = {'Neighborho' : 'Neighborhood'})
# Convert the 'Neighborhood' column to string values and then apply string methods
neighborhood_estimates['Neighborhood'] = neighborhood_estimates['Neighborhood'].fillna('').apply(str)
neighborhood_estimates['Neighborhood'] = neighborhood_estimates['Neighborhood'].str.strip().str.lower()

In [647]:
# df_enc['racecat'] = df_enc['racecat'].cat.remove_categories('missing')

race_count_enc = df_enc.pivot_table(index = 'Neighborhood', columns = 'racecat', aggfunc = 'size', fill_value = 0, observed = True)

race_count_enc = race_count_enc.rename(columns = {'white' : 'vehlvl_white', 'black' : 'vehlvl_black',
                                                      'latinx' : 'vehlvl_hispanic', 
                                                      'other' : 'vehlvl_other'})

# Reset the index so that 'neighborhood' becomes a column
race_count_enc = race_count_enc.reset_index()

#### Creating a vehicles dataframe with combined crime

In [648]:
viccombined_enc_count = pd.merge(race_vic_neigh, race_count_enc, on = 'Neighborhood', how = 'inner')
df_enc_viccombined_count = pd.merge(df_enc, viccombined_enc_count, on = 'Neighborhood', how = 'inner')
df_enc_viccombined_count.to_csv("df_enc_viccombined_count.csv", index = False)

## Adding in the neighborhood_estimates
df_enc_viccombined_count = pd.merge(df_enc_viccombined_count, neighborhood_estimates, on = 'Neighborhood', how = 'inner')
df_enc_viccombined_count.to_csv("df_enc_viccombined_count.csv", index = False)

In [649]:
col_printing(df_enc_viccombined_count)

0 anysearchperformed
1 contrabandtype_retrieved
2 search_results_descriptionb
3 individualcontraband1a
4 individualcontraband2a
5 individualcontraband3a
6 individualcontraband4a
7 individualothercontraband1a
8 individualothercontraband2a
9 individualothercontraband3a
10 individualothercontraband4a
11 vehiclecontraband11a
12 vehiclecontraband21a
13 vehiclecontraband31a
14 vehiclecontraband41a
15 vehiclecontraband12a
16 vehiclecontraband22a
17 vehiclecontraband32a
18 vehiclecontraband42a
19 vehiclecontraband13a
20 vehiclecontraband23a
21 vehiclecontraband33a
22 vehiclecontraband43a
23 vehiclecontraband14a
24 vehiclecontraband24a
25 vehiclecontraband34a
26 vehiclecontraband44a
27 vehicleothercontraband11a
28 vehicleothercontraband21a
29 vehicleothercontraband31a
30 vehicleothercontraband41a
31 vehicleothercontraband12a
32 vehicleothercontraband22a
33 vehicleothercontraband32a
34 vehicleothercontraband42a
35 vehicleothercontraband13a
36 vehicleothercontraband23a
37 vehicleothercontraband33

##### Creating a vehicles dataframe with separated crimes

In [650]:
vic_offcat1and2_enc_count = pd.merge(race_viccat_neigh_pivot, race_count_enc, on = 'Neighborhood', how = 'inner')
df_enc_vic1and2_count = pd.merge(df_enc, vic_offcat1and2_enc_count, on = 'Neighborhood', how = 'inner')
df_enc_vic1and2_count.to_csv("df_enc_vic1and2_count.csv", index = False)

## Adding in the neighborhood_estimates
df_enc_vic1and2_count = pd.merge(df_enc_vic1and2_count, neighborhood_estimates, on = 'Neighborhood', how = 'inner')
df_enc_vic1and2_count.to_csv("df_enc_vic1and2_count.csv", index = False)

## Tidying up df_enc_viccombined_count (combined offense) and df_enc_vic1and2_count (separate offenses) by removing unnecessary variables

##### Starting with the combined offenses dataframe

In [651]:
col_printing(df_enc_vic1and2_count)

0 anysearchperformed
1 contrabandtype_retrieved
2 search_results_descriptionb
3 individualcontraband1a
4 individualcontraband2a
5 individualcontraband3a
6 individualcontraband4a
7 individualothercontraband1a
8 individualothercontraband2a
9 individualothercontraband3a
10 individualothercontraband4a
11 vehiclecontraband11a
12 vehiclecontraband21a
13 vehiclecontraband31a
14 vehiclecontraband41a
15 vehiclecontraband12a
16 vehiclecontraband22a
17 vehiclecontraband32a
18 vehiclecontraband42a
19 vehiclecontraband13a
20 vehiclecontraband23a
21 vehiclecontraband33a
22 vehiclecontraband43a
23 vehiclecontraband14a
24 vehiclecontraband24a
25 vehiclecontraband34a
26 vehiclecontraband44a
27 vehicleothercontraband11a
28 vehicleothercontraband21a
29 vehicleothercontraband31a
30 vehicleothercontraband41a
31 vehicleothercontraband12a
32 vehicleothercontraband22a
33 vehicleothercontraband32a
34 vehicleothercontraband42a
35 vehicleothercontraband13a
36 vehicleothercontraband23a
37 vehicleothercontraband33

In [652]:
col_keep = list(range(0, 2)) + list(range(43, 45)) + list(range(55, 65)) + list(range(109, 117)) + list(range(130, 134)) + list(range(139, 146)) + list(range(1555, 1560)) + list(range(1812, 1822)) + list(range(1831, 1847)) + list(range(1849, 1961))

df_enc_viccombined_count = df_enc_viccombined_count.iloc[:, col_keep]
df_enc_viccombined_count.to_csv('df_enc_viccombined_count.csv', index = False)

In [653]:
df_enc_vic1and2_count.columns.tolist()

['anysearchperformed',
 'contrabandtype_retrieved',
 'search_results_descriptionb',
 'individualcontraband1a',
 'individualcontraband2a',
 'individualcontraband3a',
 'individualcontraband4a',
 'individualothercontraband1a',
 'individualothercontraband2a',
 'individualothercontraband3a',
 'individualothercontraband4a',
 'vehiclecontraband11a',
 'vehiclecontraband21a',
 'vehiclecontraband31a',
 'vehiclecontraband41a',
 'vehiclecontraband12a',
 'vehiclecontraband22a',
 'vehiclecontraband32a',
 'vehiclecontraband42a',
 'vehiclecontraband13a',
 'vehiclecontraband23a',
 'vehiclecontraband33a',
 'vehiclecontraband43a',
 'vehiclecontraband14a',
 'vehiclecontraband24a',
 'vehiclecontraband34a',
 'vehiclecontraband44a',
 'vehicleothercontraband11a',
 'vehicleothercontraband21a',
 'vehicleothercontraband31a',
 'vehicleothercontraband41a',
 'vehicleothercontraband12a',
 'vehicleothercontraband22a',
 'vehicleothercontraband32a',
 'vehicleothercontraband42a',
 'vehicleothercontraband13a',
 'vehicleo

##### Starting with the separated offenses dataframe

In [654]:
col_keep = list(range(0, 2)) + list(range(43, 45)) + list(range(55, 65)) + list(range(109, 117)) + list(range(130, 134)) + list(range(139, 146)) + list(range(1555, 1560)) + list(range(1812, 1822)) + list(range(1830, 1972))

df_enc_vic1and2_count = df_enc_vic1and2_count.iloc[:, col_keep]
df_enc_vic1and2_count.to_csv('df_enc_vic1and2_count.csv', index = False)

In [655]:
df_enc_vic1and2_count.columns.tolist()

['anysearchperformed',
 'contrabandtype_retrieved',
 'uniqueperson',
 'anypcsearch',
 'call_no',
 'TS_reason',
 'stoptype',
 'reckless',
 'wronglane',
 'trafficlight',
 'stopsign',
 'disregardsignal',
 'registration',
 'otherequipment',
 'Neighborhood',
 'top20neighborhood',
 'searchnofrisk',
 'searchperformed',
 'searcharrest',
 'encounterdate',
 'sexcat',
 'racecat',
 'morning',
 'afternoon',
 'firstevening',
 'secondevening',
 'district',
 'friskperformed',
 'searchandfrisk',
 'frisktype',
 'friskarrest',
 'top20officer',
 'top20unit',
 'youngstop',
 'blackstop',
 'latinxstop',
 'whitestop',
 'malestop',
 'male',
 'female',
 'white',
 'black',
 'latino',
 'otherrace',
 'blackwhite',
 'latinowhite',
 'young',
 'old',
 'top10neighborhood',
 'speeding',
 'otherroadsafety',
 'equipmentreg',
 'otherstopreason',
 'searchperf',
 'friskperf',
 'forciblefrisk',
 'contrabandrecd',
 'inv20officers',
 'inv20units',
 'stopresult',
 'noaction',
 'verbalwarn',
 'writtenwarn',
 'citation',
 'arrest

## Adding in more variables 
- black vehicle stop rate
- white vehicle stop rate
- black_white_total_vehstoprate
- black_veh_DI
- white_veh_DI

#### Calculating stop rates for Black and White driven vehicles (this can be used for both the combined offenses dataframe and the separated offenses dataframe)

In [656]:
new_veh_df = neighborhood_vicsus_pedenc_final.copy()

vehgrps = ['white_vehstops', 'black_vehstops']
# creating a variable that has the total pedestrian stops for black and white people
new_veh_df['black_white_total_vehstops'] = new_veh_df[vehgrps].sum(axis = 1)

# calculating white and black stop rate
new_veh_df['white_veh_stoprate'] = new_veh_df['white_vehstops']/new_veh_df['black_white_total_vehstops']
new_veh_df['black_veh_stoprate'] = new_veh_df['black_vehstops']/new_veh_df['black_white_total_vehstops']

# subset this dataframe for merging later
new_veh_df = new_veh_df[['white_veh_stoprate', 'black_veh_stoprate', 'black_white_total_vehstops', 'Neighborhood']]

#### Population Proportion for Black and White folks by neighborhood
We can use the dataframe created from before, new_pop_df

## Calculating the non-RAD Disparity Index for Vehicle Stops
This is the stop rate of a given group divided by population proportion of that given group

##### For the combined offenses dataframe

In [657]:
disparity_enc_combinedoff_df = pd.merge(new_veh_df, viccombined_enc_count, on = 'Neighborhood', how = 'inner')
disparity_enc_combinedoff_df = pd.merge(disparity_enc_combinedoff_df, new_pop_df, on = 'Neighborhood', how = 'inner')

#### For the separated offenses dataframe

In [658]:
disparity_enc_cat1and2_df = pd.merge(new_veh_df, vic_offcat1and2_enc_count, on = 'Neighborhood', how = 'inner')
disparity_enc_cat1and2_df = pd.merge(disparity_enc_cat1and2_df, new_pop_df, on = 'Neighborhood', how = 'inner')

In [659]:
col_printing(df_enc_viccombined_count)

0 anysearchperformed
1 contrabandtype_retrieved
2 uniqueperson
3 anypcsearch
4 call_no
5 TS_reason
6 stoptype
7 reckless
8 wronglane
9 trafficlight
10 stopsign
11 disregardsignal
12 registration
13 otherequipment
14 Neighborhood
15 top20neighborhood
16 searchnofrisk
17 searchperformed
18 searcharrest
19 encounterdate
20 sexcat
21 racecat
22 morning
23 afternoon
24 firstevening
25 secondevening
26 district
27 friskperformed
28 searchandfrisk
29 frisktype
30 friskarrest
31 top20officer
32 top20unit
33 youngstop
34 blackstop
35 latinxstop
36 whitestop
37 malestop
38 male
39 female
40 white
41 black
42 latino
43 otherrace
44 blackwhite
45 latinowhite
46 young
47 old
48 speeding
49 otherroadsafety
50 equipmentreg
51 otherstopreason
52 searchperf
53 friskperf
54 forciblefrisk
55 contrabandrecd
56 inv20officers
57 inv20units
58 stopresult
59 noaction
60 verbalwarn
61 writtenwarn
62 citation
63 arrest
64 AI_AN_genviccnt
65 Asian_genviccnt
66 NH_PI_genviccnt
67 black_genviccnt
68 hispanic_genvi

In [660]:
col_printing(df_enc_vic1and2_count)

0 anysearchperformed
1 contrabandtype_retrieved
2 uniqueperson
3 anypcsearch
4 call_no
5 TS_reason
6 stoptype
7 reckless
8 wronglane
9 trafficlight
10 stopsign
11 disregardsignal
12 registration
13 otherequipment
14 Neighborhood
15 top20neighborhood
16 searchnofrisk
17 searchperformed
18 searcharrest
19 encounterdate
20 sexcat
21 racecat
22 morning
23 afternoon
24 firstevening
25 secondevening
26 district
27 friskperformed
28 searchandfrisk
29 frisktype
30 friskarrest
31 top20officer
32 top20unit
33 youngstop
34 blackstop
35 latinxstop
36 whitestop
37 malestop
38 male
39 female
40 white
41 black
42 latino
43 otherrace
44 blackwhite
45 latinowhite
46 young
47 old
48 top10neighborhood
49 speeding
50 otherroadsafety
51 equipmentreg
52 otherstopreason
53 searchperf
54 friskperf
55 forciblefrisk
56 contrabandrecd
57 inv20officers
58 inv20units
59 stopresult
60 noaction
61 verbalwarn
62 writtenwarn
63 citation
64 arrest
65 citation_warning
66 racenew
67 index
68 AI_AN_offcat1_viccnt
69 AI_AN

## Merging Disparity Index Back into the Overall Vehicles dataframe

In [662]:
df_enc_viccombined_count.drop(df_enc_viccombined_count.columns[64:68], axis=1, inplace=True)
df_enc_vic1and2_count.drop(df_enc_vic1and2_count.columns[65:80], axis=1, inplace=True)

In [663]:
col_printing(df_enc_vic1and2_count)

0 anysearchperformed
1 contrabandtype_retrieved
2 uniqueperson
3 anypcsearch
4 call_no
5 TS_reason
6 stoptype
7 reckless
8 wronglane
9 trafficlight
10 stopsign
11 disregardsignal
12 registration
13 otherequipment
14 Neighborhood
15 top20neighborhood
16 searchnofrisk
17 searchperformed
18 searcharrest
19 encounterdate
20 sexcat
21 racecat
22 morning
23 afternoon
24 firstevening
25 secondevening
26 district
27 friskperformed
28 searchandfrisk
29 frisktype
30 friskarrest
31 top20officer
32 top20unit
33 youngstop
34 blackstop
35 latinxstop
36 whitestop
37 malestop
38 male
39 female
40 white
41 black
42 latino
43 otherrace
44 blackwhite
45 latinowhite
46 young
47 old
48 top10neighborhood
49 speeding
50 otherroadsafety
51 equipmentreg
52 otherstopreason
53 searchperf
54 friskperf
55 forciblefrisk
56 contrabandrecd
57 inv20officers
58 inv20units
59 stopresult
60 noaction
61 verbalwarn
62 writtenwarn
63 citation
64 arrest
65 SHAPE_STAr
66 SHAPE_STLe
67 geometry
68 Population
69 totalpop_avg
70

#### For the combined offenses dataframe

In [664]:
Vehicle_CombinedOff_RegDF = pd.merge(df_enc_viccombined_count, disparity_enc_combinedoff_df, on = 'Neighborhood', how = 'inner')
Vehicle_CombinedOff_RegDF.to_csv('Vehicle_CombinedOff_RegDF.csv', index = False)

#### For the separated offenses dataframe

In [204]:
col_printing(df_enc_vic1and2_count)
# col_printing(disparity_enc_cat1and2_df)

0 anysearchperformed
1 contrabandtype_retrieved
2 uniqueperson
3 anypcsearch
4 call_no
5 TS_reason
6 stoptype
7 reckless
8 wronglane
9 trafficlight
10 stopsign
11 disregardsignal
12 registration
13 otherequipment
14 Neighborhood
15 top20neighborhood
16 searchnofrisk
17 searchperformed
18 searcharrest
19 encounterdate
20 sexcat
21 racecat
22 morning
23 afternoon
24 firstevening
25 secondevening
26 district
27 friskperformed
28 searchandfrisk
29 frisktype
30 friskarrest
31 top20officer
32 top20unit
33 youngstop
34 blackstop
35 latinxstop
36 whitestop
37 malestop
38 male
39 female
40 white
41 black
42 latino
43 otherrace
44 blackwhite
45 latinowhite
46 young
47 old
48 top10neighborhood
49 speeding
50 otherroadsafety
51 equipmentreg
52 otherstopreason
53 searchperf
54 friskperf
55 forciblefrisk
56 contrabandrecd
57 inv20officers
58 inv20units
59 stopresult
60 racenew
61 black_offcat2_vicrate
62 white_offcat2_vicrate
63 vehlvl_white
64 vehlvl_black
65 vehlvl_hispanic
66 vehlvl_other
67 OBJE

In [665]:
Vehicle_SepOff_RegDF = pd.merge(df_enc_vic1and2_count, disparity_enc_cat1and2_df, on = 'Neighborhood', how = 'inner')
Vehicle_SepOff_RegDF.to_csv('Vehicle_SepOff_RegDF.csv', index = False)

In [666]:
Vehicle_SepOff_RegDF.columns.tolist()

['anysearchperformed',
 'contrabandtype_retrieved',
 'uniqueperson',
 'anypcsearch',
 'call_no',
 'TS_reason',
 'stoptype',
 'reckless',
 'wronglane',
 'trafficlight',
 'stopsign',
 'disregardsignal',
 'registration',
 'otherequipment',
 'Neighborhood',
 'top20neighborhood',
 'searchnofrisk',
 'searchperformed',
 'searcharrest',
 'encounterdate',
 'sexcat',
 'racecat',
 'morning',
 'afternoon',
 'firstevening',
 'secondevening',
 'district',
 'friskperformed',
 'searchandfrisk',
 'frisktype',
 'friskarrest',
 'top20officer',
 'top20unit',
 'youngstop',
 'blackstop',
 'latinxstop',
 'whitestop',
 'malestop',
 'male',
 'female',
 'white',
 'black',
 'latino',
 'otherrace',
 'blackwhite',
 'latinowhite',
 'young',
 'old',
 'top10neighborhood',
 'speeding',
 'otherroadsafety',
 'equipmentreg',
 'otherstopreason',
 'searchperf',
 'friskperf',
 'forciblefrisk',
 'contrabandrecd',
 'inv20officers',
 'inv20units',
 'stopresult',
 'noaction',
 'verbalwarn',
 'writtenwarn',
 'citation',
 'arrest

#### Delete extra columns for vehicle files

In [ ]:
Vehicle_CombinedOff_RegDF = pd.read_csv('C:/Users/blah/blah/Documents/Data/WI_RAD/Vehicle_CombinedOff_RegDF.csv', low_memory = False)
Vehicle_SepOff_RegDF = pd.read_csv('C:/Users/blah/blah/Data/WI_RAD/Vehicle_SepOff_RegDF.csv', low_memory = False)

In [176]:
Vehicle_CombinedOff_RegDF.drop(Vehicle_CombinedOff_RegDF.columns[59:64], axis=1, inplace=True)
Vehicle_SepOff_RegDF.drop(Vehicle_SepOff_RegDF.columns[59:64], axis=1, inplace=True)

Vehicle_CombinedOff_RegDF = Vehicle_CombinedOff_RegDF.rename(columns = {'white_genvicrate_y' : 'white_genvicrate',
                                                                        'vehlvl_white_y' : 'vehlvl_white',
                                                                        'vehlvl_black_y' : 'vehlvl_black',
                                                                        'vehlvl_hispanic_y' : 'vehlvl_hispanic',
                                                                        'vehlvl_other_y' : 'vehlvl_other'})

Vehicle_SepOff_RegDF = Vehicle_SepOff_RegDF.rename(columns = {'white_genvicrate_y' : 'white_genvicrate',
                                                                        'vehlvl_white_y' : 'vehlvl_white',
                                                                        'vehlvl_black_y' : 'vehlvl_black',
                                                                        'vehlvl_hispanic_y' : 'vehlvl_hispanic',
                                                                        'vehlvl_other_y' : 'vehlvl_other'})


In [583]:
# CALCULATING DISPARITY INDEX
Vehicle_CombinedOff_RegDF['white_veh_combinedoff_DI'] = Vehicle_CombinedOff_RegDF['white_veh_stoprate'] / Vehicle_CombinedOff_RegDF['whitepop_prop']
Vehicle_CombinedOff_RegDF['black_veh_combinedoff_DI'] = Vehicle_CombinedOff_RegDF['black_veh_stoprate'] / Vehicle_CombinedOff_RegDF['blackpop_prop']


Vehicle_SepOff_RegDF['white_veh_sepoff_DI'] = Vehicle_SepOff_RegDF['white_veh_stoprate'] / Vehicle_SepOff_RegDF['whitepop_prop']
Vehicle_SepOff_RegDF['black_veh_sepoff_DI'] = Vehicle_SepOff_RegDF['black_veh_stoprate'] / Vehicle_SepOff_RegDF['blackpop_prop']

In [584]:
Vehicle_CombinedOff_RegDF.to_csv('Vehicle_CombinedOff_RegDF.csv', index = False)
Vehicle_SepOff_RegDF.to_csv('Vehicle_SepOff_RegDF.csv', index = False)

# Creating the RAD Index Dataframe for Category 1 Crimes

In [179]:
race_viccat_neigh_pivot.head()

Race_Offense,index,Neighborhood,AI_AN_offcat1_viccnt,AI_AN_offcat2_viccnt,Asian_offcat1_viccnt,Asian_offcat2_viccnt,NH_PI_offcat1_viccnt,NH_PI_offcat2_viccnt,black_offcat1_viccnt,black_offcat2_viccnt,...,other_offcat1_viccnt,other_offcat2_viccnt,white_offcat1_viccnt,white_offcat2_viccnt,total_cat1viccnt,total_cat2viccnt,black_offcat1_vicrate,white_offcat1_vicrate,black_offcat2_vicrate,white_offcat2_vicrate
0,0,alcott park,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,...,0.0,0.0,0.0,3.0,1.0,5.0,1.000000,0.000000,0.400000,0.600000
1,1,alverno,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,...,0.0,0.0,4.0,8.0,9.0,14.0,0.111111,0.444444,0.214286,0.571429
2,2,amani,0.0,2.0,0.0,0.0,0.0,0.0,169.0,93.0,...,0.0,0.0,5.0,5.0,177.0,103.0,0.954802,0.028249,0.902913,0.048544
3,3,arlington gardens,0.0,0.0,0.0,1.0,0.0,0.0,58.0,56.0,...,0.0,0.0,12.0,11.0,73.0,71.0,0.794521,0.164384,0.788732,0.154930
4,4,arlington heights,1.0,1.0,5.0,0.0,0.0,0.0,318.0,221.0,...,0.0,0.0,27.0,19.0,360.0,246.0,0.883333,0.075000,0.898374,0.077236


In [180]:
RAD_cat1_df = race_viccat_neigh_pivot.copy()

In [181]:
RAD_cat1_df = RAD_cat1_df.drop(columns = ['AI_AN_offcat2_viccnt', 'Asian_offcat2_viccnt',
                                          'NH_PI_offcat2_viccnt', 'black_offcat2_viccnt',
                                          'hispanic_offcat2_viccnt', 'other_offcat2_viccnt',
                                          'white_offcat2_viccnt', 'total_cat1viccnt',
                                          'total_cat2viccnt', 'black_offcat1_vicrate', 
                                          'white_offcat1_vicrate', 'black_offcat2_vicrate',
                                          'white_offcat2_vicrate'])

In [183]:
RAD_cat1_df = pd.merge(RAD_cat1_df, neighborhood_vicsus_pedenc_final, on = 'Neighborhood', how = 'inner')

In [ ]:
# subsection of RAD_cat1_df, extracting only homicide rates
homicide_cols = ['Neighborhood', 'AI_AN_total_homicide_x', 'Asian_total_homicide_x', 'hispanic_total_homicide_x', 'black_total_homicide_x', 'white_total_homicide_x']
homicide_rates = RAD_cat1_df[homicide_cols]

homicide_rates = homicide_rates.rename(columns = {'AI_AN_total_homicide_x' : 'AI_AN_total_homicide',
                                                  'Asian_total_homicide_x' : 'Asian_total_homicide',
                                                  'hispanic_total_homicide_x' : 'hispanic_total_homicide',
                                                  'black_total_homicide_x' : 'black_total_homicide',
                                                  'white_total_homicide_x' : 'white_total_homicide'})

homicide_rates['total_homicide'] = homicide_rates['AI_AN_total_homicide'] + homicide_rates['Asian_total_homicide'] + homicide_rates['hispanic_total_homicide'] + homicide_rates['black_total_homicide'] + homicide_rates['white_total_homicide']

In [197]:
homicide_rates.columns.tolist()

['Neighborhood',
 'AI_AN_total_homicide',
 'Asian_total_homicide',
 'hispanic_total_homicide',
 'black_total_homicide',
 'white_total_homicide']

In [199]:
homicide_rates.to_csv('homicide_rates_only.csv', index = False)

In [181]:
RAD_cat1_final = RAD_cat1_df[['Neighborhood', 'totalpop_avg', 'white_not_hisp_avg',
                              'black_not_hisp_avg', 'white_pedstops', 'black_pedstops',
                              'latinx_pedstops', 'other_pedstops', 'white_vehstops',
                              'black_vehstops', 'latinx_vehstops', 'other_vehstops',
                              'black_Part I_vic', 'white_Part I_vic', 'totalmale_avg',
                              'total_under5_avg', 'total10to14_avg', 'total_5to9_avg',
                              'total_15to19_avg',
                              'total_20to24_avg', 'total_25to29_avg', 'total_60to64_avg',
                              'total_65to69_avg', 'total_70to74_avg', 'total_75to79_avg',
                              'total_80to84_avg', 'total_85andolder_avg', 'total_hispanic_avg',
                              'AI_AN_Part I_vic', 'Asian_Part I_vic', 
                              'NH_PI_Part I_vic', 'hispanic_Part I_vic', 'other_Part I_vic']].copy()

RAD_cat1_final['percent_blackpop'] = RAD_cat1_final['black_not_hisp_avg'] / RAD_cat1_final['totalpop_avg'].replace(0, np.nan)
RAD_cat1_final['percent_whitepop'] = RAD_cat1_final['white_not_hisp_avg'] / RAD_cat1_final['totalpop_avg'].replace(0, np.nan)
RAD_cat1_final['total_PartI_vic'] = RAD_cat1_final['black_Part I_vic'] + RAD_cat1_final['white_Part I_vic'] + RAD_cat1_final['AI_AN_Part I_vic'] + RAD_cat1_final['Asian_Part I_vic'] + RAD_cat1_final['NH_PI_Part I_vic'] + RAD_cat1_final['hispanic_Part I_vic'] + RAD_cat1_final['other_Part I_vic']

In [182]:
col_printing(RAD_cat1_final)

0 Neighborhood
1 totalpop_avg
2 white_not_hisp_avg
3 black_not_hisp_avg
4 white_pedstops
5 black_pedstops
6 latinx_pedstops
7 other_pedstops
8 white_vehstops
9 black_vehstops
10 latinx_vehstops
11 other_vehstops
12 black_Part I_vic
13 white_Part I_vic
14 totalmale_avg
15 total_under5_avg
16 total10to14_avg
17 total_5to9_avg
18 total_15to19_avg
19 total_20to24_avg
20 total_25to29_avg
21 total_60to64_avg
22 total_65to69_avg
23 total_70to74_avg
24 total_75to79_avg
25 total_80to84_avg
26 total_85andolder_avg
27 total_hispanic_avg
28 AI_AN_Part I_vic
29 Asian_Part I_vic
30 NH_PI_Part I_vic
31 hispanic_Part I_vic
32 other_Part I_vic
33 percent_blackpop
34 percent_whitepop
35 total_PartI_vic


In [183]:
RAD_cat2_final = RAD_cat1_df[['Neighborhood', 'totalpop_avg', 'white_not_hisp_avg',
                              'black_not_hisp_avg', 'white_pedstops', 'black_pedstops',
                              'latinx_pedstops', 'other_pedstops', 'white_vehstops',
                              'black_vehstops', 'latinx_vehstops', 'other_vehstops',
                              'black_Part II_vic', 'white_Part II_vic', 
                              'AI_AN_Part II_vic', 'Asian_Part II_vic', 
                              'NH_PI_Part II_vic', 'hispanic_Part II_vic', 'other_Part II_vic',
                              'totalmale_avg', 'total_15to19_avg',
                              'total_under5_avg', 'total10to14_avg', 'total_5to9_avg',
                              'total_20to24_avg', 'total_25to29_avg', 'total_60to64_avg',
                              'total_65to69_avg', 'total_70to74_avg', 'total_75to79_avg',
                              'total_80to84_avg', 'total_85andolder_avg', 'total_hispanic_avg']]

RAD_cat2_final = RAD_cat2_final.copy()
RAD_cat2_final.loc[:, 'percent_blackpop'] = RAD_cat2_final['black_not_hisp_avg'] / RAD_cat2_final['totalpop_avg'].replace(0, np.nan)
RAD_cat2_final.loc[:, 'percent_whitepop'] = RAD_cat2_final['white_not_hisp_avg'] / RAD_cat2_final['totalpop_avg'].replace(0, np.nan)
RAD_cat2_final.loc[:, 'total_PartII_vic'] = RAD_cat2_final['black_Part II_vic'] + RAD_cat2_final['white_Part II_vic'] + RAD_cat2_final['AI_AN_Part II_vic'] + RAD_cat2_final['Asian_Part II_vic'] + RAD_cat2_final['NH_PI_Part II_vic'] + RAD_cat2_final['hispanic_Part II_vic'] + RAD_cat2_final['other_Part II_vic']


In [185]:
RAD_cat3_final = pd.merge(RAD_cat1_final, RAD_cat2_final, on = 'Neighborhood', how = 'outer')
RAD_cat3_final = RAD_cat3_final.drop(columns = {'totalpop_avg_y', 'white_not_hisp_avg_y', 'black_not_hisp_avg_y', 
                                                'white_pedstops_y', 'black_pedstops_y', 'percent_blackpop_y', 
                                                'percent_whitepop_y', 'latinx_pedstops_y', 'other_pedstops_y',
                                                'white_vehstops_y', 'black_vehstops_y', 
                                                'latinx_vehstops_y', 'other_vehstops_y',
                                                'totalmale_avg_y', 'total_under5_avg_y', 'total10to14_avg_y',
                                                'total_5to9_avg_y', 'total_20to24_avg_y', 'total_25to29_avg_y',
                                                'total_60to64_avg_y', 'total_65to69_avg_y', 'total_15to19_avg_y',
                                                'total_70to74_avg_y', 'total_75to79_avg_y', 'total_80to84_avg_y',
                                                'total_85andolder_avg_y', 'total_hispanic_avg_y'})

RAD_cat3_final = RAD_cat3_final.rename(columns = {'totalpop_avg_x' : 'totalpop_avg',
                                                  'white_not_hisp_avg_x' : 'white_not_hisp_avg',
                                                  'black_not_hisp_avg_x' : 'black_not_hisp_avg',
                                                  'white_pedstops_x' : 'white_pedstops',
                                                  'black_pedstops_x' : 'black_pedstops',
                                                  'latinx_pedstops_x' : 'latinx_pedstops',
                                                  'other_pedstops_x' : 'other_pedstops',
                                                  'white_vehstops_x' : 'white_vehstops',
                                                  'black_vehstops_x' : 'black_vehstops',
                                                  'latinx_vehstops_x' : 'latinx_vehstops',
                                                  'other_vehstops_x' : 'other_vehstops',
                                                  'percent_blackpop_x' : 'percent_blackpop',
                                                  'percent_whitepop_x' : 'percent_whitepop',
                                                  'totalmale_avg_x' : 'totalmale_avg',
                                                  'total_under5_avg_x' : 'total_under5_avg',
                                                  'total_15to19_avg_x' : 'total_15to19_avg',
                                                  'total10to14_avg_x' : 'total10to14_avg',
                                                  'total_5to9_avg_x' : 'total_5to9_avg',
                                                  'total_20to24_avg_x' : 'total_20to24_avg',
                                                  'total_25to29_avg_x' : 'total_25to29_avg',
                                                  'total_60to64_avg_x' : 'total_60to64_avg',
                                                  'total_65to69_avg_x' : 'total_65to69_avg',
                                                  'total_70to74_avg_x' : 'total_70to74_avg',
                                                  'total_75to79_avg_x' : 'total_75to79_avg',
                                                  'total_80to84_avg_x' : 'total_80to84_avg',
                                                  'total_85andolder_avg_x' : 'total_85andolder_avg',
                                                  'total_hispanic_avg_x' : 'total_hispanic_avg'})



In [186]:
RAD_cat3_final.columns

Index(['Neighborhood', 'totalpop_avg', 'white_not_hisp_avg',
       'black_not_hisp_avg', 'white_pedstops', 'black_pedstops',
       'latinx_pedstops', 'other_pedstops', 'white_vehstops', 'black_vehstops',
       'latinx_vehstops', 'other_vehstops', 'black_Part I_vic',
       'white_Part I_vic', 'totalmale_avg', 'total_under5_avg',
       'total10to14_avg', 'total_5to9_avg', 'total_15to19_avg',
       'total_20to24_avg', 'total_25to29_avg', 'total_60to64_avg',
       'total_65to69_avg', 'total_70to74_avg', 'total_75to79_avg',
       'total_80to84_avg', 'total_85andolder_avg', 'total_hispanic_avg',
       'AI_AN_Part I_vic', 'Asian_Part I_vic', 'NH_PI_Part I_vic',
       'hispanic_Part I_vic', 'other_Part I_vic', 'percent_blackpop',
       'percent_whitepop', 'total_PartI_vic', 'black_Part II_vic',
       'white_Part II_vic', 'AI_AN_Part II_vic', 'Asian_Part II_vic',
       'NH_PI_Part II_vic', 'hispanic_Part II_vic', 'other_Part II_vic',
       'total_PartII_vic'],
      dtype='object'

In [187]:
# create a new variable for total vic counts for part I and II
RAD_cat3_final['total_PartI_II_combined'] = RAD_cat3_final['total_PartI_vic'] + RAD_cat3_final['total_PartII_vic']
RAD_cat3_final['black_total_vic'] = RAD_cat3_final['black_Part I_vic'] + RAD_cat3_final['black_Part II_vic']
RAD_cat3_final['white_total_vic'] = RAD_cat3_final['white_Part I_vic'] + RAD_cat3_final['white_Part II_vic']

In [188]:
# creating stop rates and RAD
# PEDESTRIAN STOPS
RAD_cat3_final['black_ped_stoprate'] = (RAD_cat3_final['black_pedstops'] / RAD_cat3_final['black_not_hisp_avg']) * 1000
RAD_cat3_final['white_ped_stoprate'] = (RAD_cat3_final['white_pedstops'] / RAD_cat3_final['white_not_hisp_avg']) * 1000
RAD_cat3_final['black_vicrate'] = (RAD_cat3_final['black_total_vic'] / RAD_cat3_final['black_not_hisp_avg']) * 1000
RAD_cat3_final['white_vicrate'] = (RAD_cat3_final['white_total_vic'] / RAD_cat3_final['white_not_hisp_avg']) * 1000
RAD_cat3_final['black_ped_ratio'] = RAD_cat3_final['black_ped_stoprate'] / RAD_cat3_final['black_vicrate']
RAD_cat3_final['white_ped_ratio'] = RAD_cat3_final['white_ped_stoprate'] / RAD_cat3_final['white_vicrate']
RAD_cat3_final['ped_RAD'] = RAD_cat3_final['black_ped_ratio'] / RAD_cat3_final['white_ped_ratio']

# VEHICLE STOPS
RAD_cat3_final['black_veh_stoprate'] = (RAD_cat3_final['black_vehstops'] / RAD_cat3_final['black_not_hisp_avg']) * 1000
RAD_cat3_final['white_veh_stoprate'] = (RAD_cat3_final['white_vehstops'] / RAD_cat3_final['white_not_hisp_avg']) * 1000
RAD_cat3_final['black_veh_ratio'] = RAD_cat3_final['black_veh_stoprate'] / RAD_cat3_final['black_vicrate']
RAD_cat3_final['white_veh_ratio'] = RAD_cat3_final['white_veh_stoprate'] / RAD_cat3_final['white_vicrate']
RAD_cat3_final['veh_RAD'] = RAD_cat3_final['black_veh_ratio'] / RAD_cat3_final['white_veh_ratio']

In [189]:
# Define the file path
file_path = "RAD_seperatedcat3.xlsx"

# Save DataFrames to separate sheets
with pd.ExcelWriter(file_path, engine = "xlsxwriter") as writer:
    RAD_cat1_final.to_excel(writer, sheet_name = "Category1", index = False)
    RAD_cat2_final.to_excel(writer, sheet_name ="Category2", index = False)
    RAD_cat3_final.to_excel(writer, sheet_name = "Combined", index = False)